In [52]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# import lightgbm as lgb

# import xgboost as xgb

In [53]:
df_train_set = pd.read_csv('train_aggregated.csv', low_memory=False)
test = pd.read_csv('test_questions.csv', low_memory=False)

In [54]:
df_train_set.drop(['ride_id'], axis=1, inplace=True) #ride_id is unnecessary in training set
df_train_set["travel_date"] = pd.to_datetime(df_train_set["travel_date"],infer_datetime_format=True)
df_train_set["travel_date"] = df_train_set["travel_date"].dt.dayofweek #change the full date to day of week

# df_train_set["car_type"] = pd.Categorical(df_train_set["car_type"])
# car_type_categories = df_train_set.car_type.cat.categories
# df_train_set["car_type"] = df_train_set.car_type.cat.codes

# df_train_set["travel_from"] = pd.Categorical(df_train_set["travel_from"])
# travel_from_categories = df_train_set.travel_from.cat.categories
# df_train_set["travel_from"] = df_train_set.travel_from.cat.codes

#express travel time in minutes
df_train_set["travel_time"] = df_train_set["travel_time"].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

df_train_set.head()

,travel_date,travel_time,travel_from,car_type,max_capacity,number_of_tickets
0,1,435,Migori,Bus,49,1.0
1,6,432,Migori,Bus,49,1.0
2,6,425,Keroka,Bus,49,1.0
3,0,430,Homa Bay,Bus,49,5.0
4,0,432,Migori,Bus,49,31.0


In [55]:
df_train_set.drop('car_type', inplace = True, axis = 1)
test.drop('car_type', axis = 1 , inplace = True)

In [56]:
X = df_train_set.drop(["number_of_tickets"], axis=1)
y = df_train_set.number_of_tickets
test_ID = test["ride_id"] 



In [57]:
#Test set
test["travel_date"] = pd.to_datetime(test["travel_date"],infer_datetime_format=True)
test["travel_date"] = test["travel_date"].dt.dayofweek

# test["car_type"] = pd.Categorical(test["car_type"], categories=car_type_categories)
# test["car_type"] = test.car_type.cat.codes

# test["travel_from"] = pd.Categorical(test["travel_from"], categories=travel_from_categories)
# test["travel_from"] = test.travel_from.cat.codes

test["travel_time"] = test["travel_time"].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

test.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,max_capacity
0,247,0,426,Kisii,Nairobi,49
1,256,6,668,Kisii,Nairobi,11
2,275,4,300,Kisii,Nairobi,11
3,285,4,550,Kisii,Nairobi,11
4,286,4,560,Kisii,Nairobi,11


In [58]:
df_train_set

,travel_date,travel_time,travel_from,max_capacity,number_of_tickets
0,1,435,Migori,49,1.0
1,6,432,Migori,49,1.0
2,6,425,Keroka,49,1.0
3,0,430,Homa Bay,49,5.0
4,0,432,Migori,49,31.0
5,0,429,Homa Bay,49,26.0
6,4,310,Kisii,11,1.0
7,4,590,Kisii,11,1.0
8,4,426,Kisii,49,2.0
9,4,360,Kisii,11,1.0


In [59]:
test

,ride_id,travel_date,travel_time,travel_from,travel_to,max_capacity
0,247,0,426,Kisii,Nairobi,49
1,256,6,668,Kisii,Nairobi,11
2,275,4,300,Kisii,Nairobi,11
3,285,4,550,Kisii,Nairobi,11
4,286,4,560,Kisii,Nairobi,11
5,287,4,422,Kisii,Nairobi,49
6,288,4,570,Kisii,Nairobi,11
7,292,4,1143,Kisii,Nairobi,49
8,298,4,540,Kisii,Nairobi,11
9,302,4,520,Kisii,Nairobi,11


In [60]:
df_train_set.head()

,travel_date,travel_time,travel_from,max_capacity,number_of_tickets
0,1,435,Migori,49,1.0
1,6,432,Migori,49,1.0
2,6,425,Keroka,49,1.0
3,0,430,Homa Bay,49,5.0
4,0,432,Migori,49,31.0


In [61]:
# def add_datepart(df, fldname, drop=True, time=True):
#     """add_datepart converts a column of df from a datetime64 to many columns containing
#     the information from the date. This applies changes inplace.
#     Parameters:
#    """
#     import re
#     fld = df[fldname]
#     fld_dtype = fld.dtype
#     if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
#         fld_dtype = np.datetime64

#     if not np.issubdtype(fld_dtype, np.datetime64):
#         df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
#     targ_pre = re.sub('[Dd]ate$', '', fldname)
#     attr = ['Week', 'Day', 'Dayofweek', 'Dayofyear',
#             'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
#     if time: attr = attr + ['Hour', 'Minute']
#     for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
#     df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
#     if drop: df.drop(fldname, axis=1, inplace=True)

In [62]:
test.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,max_capacity
0,247,0,426,Kisii,Nairobi,49
1,256,6,668,Kisii,Nairobi,11
2,275,4,300,Kisii,Nairobi,11
3,285,4,550,Kisii,Nairobi,11
4,286,4,560,Kisii,Nairobi,11


In [63]:
df_train_set = df_train_set.rename(columns={'370.4': 'travel'})
test = test.rename(columns={'305.1': 'travel'})

In [64]:
df_train_set.head()

,travel_date,travel_time,travel_from,max_capacity,number_of_tickets
0,1,435,Migori,49,1.0
1,6,432,Migori,49,1.0
2,6,425,Keroka,49,1.0
3,0,430,Homa Bay,49,5.0
4,0,432,Migori,49,31.0


In [65]:
test.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,max_capacity
0,247,0,426,Kisii,Nairobi,49
1,256,6,668,Kisii,Nairobi,11
2,275,4,300,Kisii,Nairobi,11
3,285,4,550,Kisii,Nairobi,11
4,286,4,560,Kisii,Nairobi,11


In [66]:
X = df_train_set.drop(["number_of_tickets"], axis=1)
y = df_train_set.number_of_tickets
test_ID = test["ride_id"] 

In [67]:
test.drop(['travel_to', 'ride_id'], axis=1, inplace=True)


In [68]:
test.shape

(1111, 4)

In [69]:
df_train_set.shape

(6249, 5)

In [70]:
X.head()

,travel_date,travel_time,travel_from,max_capacity
0,1,435,Migori,49
1,6,432,Migori,49
2,6,425,Keroka,49
3,0,430,Homa Bay,49
4,0,432,Migori,49


In [71]:
ntrain = X.shape[0]
ntest = test.shape[0]

all_data = pd.concat((X, test)).reset_index(drop=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (7360, 4)


In [72]:
ntest

1111

In [73]:
all_data = pd.get_dummies(all_data)

#Get the new dataset
train = all_data[:ntrain]
test = all_data[ntrain:]

In [75]:
# add_datepart(X, fldname='travel_date')
# add_datepart(test, fldname='travel_date')

In [76]:
train.head()

,travel_date,travel_time,max_capacity,travel_from_Awendo,travel_from_Homa Bay,travel_from_Kehancha,travel_from_Kendu Bay,travel_from_Keroka,travel_from_Keumbu,travel_from_Kijauri,travel_from_Kisii,travel_from_Mbita,travel_from_Migori,travel_from_Ndhiwa,travel_from_Nyachenge,travel_from_Oyugis,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori
0,1,435,49,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,432,49,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,6,425,49,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,430,49,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,432,49,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [77]:
test.head()

,travel_date,travel_time,max_capacity,travel_from_Awendo,travel_from_Homa Bay,travel_from_Kehancha,travel_from_Kendu Bay,travel_from_Keroka,travel_from_Keumbu,travel_from_Kijauri,travel_from_Kisii,travel_from_Mbita,travel_from_Migori,travel_from_Ndhiwa,travel_from_Nyachenge,travel_from_Oyugis,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori
6249,0,426,49,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6250,6,668,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6251,4,300,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6252,4,550,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6253,4,560,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [78]:
#model_lgb = lgb.LGBMRegressor(n_estimators=200,learning_rate=0.1, n_jobs=-1, seed =2018)
#model_xgb  = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, subsample=0.5, seed =2018)
model = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=3, max_features='sqrt', min_samples_leaf= 15, min_samples_split= 10,  loss='huber',subsample=0.8, random_state = 3)


In [79]:
#test.head()

In [80]:
test = test.reset_index()

In [81]:
test.drop('index',inplace = True , axis = 1)

In [82]:
test.head()

,travel_date,travel_time,max_capacity,travel_from_Awendo,travel_from_Homa Bay,travel_from_Kehancha,travel_from_Kendu Bay,travel_from_Keroka,travel_from_Keumbu,travel_from_Kijauri,travel_from_Kisii,travel_from_Mbita,travel_from_Migori,travel_from_Ndhiwa,travel_from_Nyachenge,travel_from_Oyugis,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori
0,0,426,49,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,6,668,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,4,300,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,4,550,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,4,560,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [83]:
train.head()

,travel_date,travel_time,max_capacity,travel_from_Awendo,travel_from_Homa Bay,travel_from_Kehancha,travel_from_Kendu Bay,travel_from_Keroka,travel_from_Keumbu,travel_from_Kijauri,travel_from_Kisii,travel_from_Mbita,travel_from_Migori,travel_from_Ndhiwa,travel_from_Nyachenge,travel_from_Oyugis,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori
0,1,435,49,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,432,49,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,6,425,49,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,430,49,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,432,49,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [84]:
test.head()

,travel_date,travel_time,max_capacity,travel_from_Awendo,travel_from_Homa Bay,travel_from_Kehancha,travel_from_Kendu Bay,travel_from_Keroka,travel_from_Keumbu,travel_from_Kijauri,travel_from_Kisii,travel_from_Mbita,travel_from_Migori,travel_from_Ndhiwa,travel_from_Nyachenge,travel_from_Oyugis,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori
0,0,426,49,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,6,668,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,4,300,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,4,550,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,4,560,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [85]:
test.head()

,travel_date,travel_time,max_capacity,travel_from_Awendo,travel_from_Homa Bay,travel_from_Kehancha,travel_from_Kendu Bay,travel_from_Keroka,travel_from_Keumbu,travel_from_Kijauri,travel_from_Kisii,travel_from_Mbita,travel_from_Migori,travel_from_Ndhiwa,travel_from_Nyachenge,travel_from_Oyugis,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori
0,0,426,49,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,6,668,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,4,300,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,4,550,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,4,560,11,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [86]:
from sklearn.cross_validation import train_test_split

In [87]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)

In [88]:
model.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             presort='auto', random_state=3, subsample=0.8, verbose=0,
             warm_start=False)

In [89]:
model.predict(X_test)

array([ 8.45031537,  8.91195899,  3.1801011 , ...,  8.19045127,
       27.4330655 ,  3.25893569])

In [91]:
pred_rf = model.predict(X_test)
print (mean_absolute_error(y_test,pred_rf))

3.7868780782090865


In [93]:
sub = model.predict(test)

In [94]:
sub

array([ 6.65607973,  7.22809756,  2.33597228, ...,  5.7869851 ,
       10.84144385, 16.88322453])

In [95]:
d = {'ride_id': test_ID, 'number_of_ticket': sub.round()}
df_predictions = pd.DataFrame(data=d)
df_predictions = df_predictions[['ride_id','number_of_ticket']]

In [96]:
df_predictions.head()

,ride_id,number_of_ticket
0,247,7.0
1,256,7.0
2,275,2.0
3,285,9.0
4,286,9.0


In [97]:
df_predictions.to_csv('gbbb.csv',index = False)

In [ ]:
%matplotlib inline

In [ ]:
import xgboost as xgb

In [ ]:
xgb.plot_importance(model_xgb)

In [ ]:
#from sklearn.grid_search import GridSearchCV

In [ ]:
#param_grid = {'learning_rate':[0.001,0.1,1,2],'n_estimators': [10, 50 ,100, 1000], 'max_depth': [1,2,4,6,8,10,12],'subsample': [0.1,0.2,0.3,0.4,0.5,1],'gamma': [1,0.1,0.01,0.001,0.0001]}

In [ ]:
grid = GridSearchCV(XGBRegressor(),param_grid,refit=True,verbose=3)

In [ ]:
#grid.fit(train,y)